In [63]:
import os
os.getcwd()

import sys 
sys.path.append(os.path.abspath(os.path.join('../')))
from langchain import LLMChain
from langchain import SagemakerEndpoint
from langchain.prompts import PromptTemplate
from langchain.llms.sagemaker_endpoint import LLMContentHandler
import os
import json

from langchain.vectorstores import Pinecone

from langchain.chains import RetrievalQAWithSourcesChain

from dotenv import load_dotenv
load_dotenv()




from config import *
from src.utils import connect_index

region = os.environ["AWS_REGION"]
endpoint_name = os.environ["FALCON_ENDPOINT_NAME"]

In [64]:
embeddings = CohereEmbeddings(model=COHERE_MODEL_NAME, cohere_api_key=COHERE_API_KEY)

In [65]:
index = connect_index(PINECONE_INDEX_NAME)
vectorstore = Pinecone(index, embeddings.embed_query, "text")

In [66]:
class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: dict) -> bytes:
        input_str = json.dumps({"inputs": prompt,
        "parameters" : {**model_kwargs}})
        return input_str.encode('utf-8')
    
    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json[0]["generated_text"]

In [67]:
template = "{content}"

prompt = PromptTemplate.from_template(template)

In [68]:
content_handler = ContentHandler()

llm=SagemakerEndpoint(
     endpoint_name=endpoint_name, 
     region_name=region, 
     model_kwargs={"max_new_tokens": 500, "top_p": 0.9, "max_length": None, "temperature":0.01},
     endpoint_kwargs={"CustomAttributes": 'accept_eula=true'},
     content_handler=content_handler,
     credentials_profile_name="fundamentl-ai"
 )

2023-08-08 13:47:31.864 Found credentials in shared credentials file: ~/.aws/credentials


In [69]:
qa = RetrievalQAWithSourcesChain.from_chain_type(llm=llm, retriever=vectorstore.as_retriever(), chain_type="refine")

In [70]:
query = "What is Semantich  Search?"
# Send question as a query to qa chain
result = qa({"question": query})

In [71]:
result

{'question': 'What is Semantich  Search?',
 'answer': '\nThe original answer is correct, but it may not be sufficient for the question. The question asks about the search engine Semantich Search, and the answer is about the STSG algorithm. To answer the question, we need to provide more information about Semantich Search.\n\nSemantich Search is a search engine that uses probabilistic principles to search for information in a corpus. It is designed to be able to handle large amounts of data and to be able to provide accurate results in a timely manner. The search engine is designed to be able to handle both P and NP-complete problems, although this is yet unproven. The search engine is designed to be able to handle large amounts of data and to be able to provide accurate results in a timely manner.\n\nThe STSG algorithm is a parsing algorithm that is used to parse a grammar. It is designed to be able to parse a grammar that is defined in a specific language. The STSG algorithm is design

In [74]:
query = "What is STSG algorithm?"
# Send question as a query to qa chain
result = qa({"question": query})

In [75]:
result

{'question': 'What is STSG algorithm?',
 'answer': '\nThe STSG algorithm is a search algorithm used in artificial intelligence to find the best solution to a problem. It is commonly used in optimization problems, such as finding the best route in a graph or the best solution to a puzzle. The algorithm uses a greedy approach to search for the best solution, where it always tries to maximize its current state. In the context of the question, the dog with no nose is the best solution, as it is the most efficient and optimal solution.',
 'sources': ''}

In [76]:
llm_chain = LLMChain(
     llm=llm,
     prompt=prompt
 )

In [77]:
llm_chain.run({"How can I travel from New York to Los Angeles?"})


"\nThere are several ways to travel from New York to Los Angeles. One option is to take a flight, which can be booked through various airlines. Another option is to take a train, such as Amtrak's California Zephyr, which departs from New York and stops in Los Angeles. You can also take a bus or drive, but these options may take longer and require more planning."